VGG16

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import time

In [0]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


def get_onehot_encoder(labels):
    values = np.array(labels).reshape(-1, 1)
    # integer-binary encode
    onehot_encoder = OneHotEncoder(sparse=False, dtype=np.uint8, categories='auto')
    onehot_encoder.fit(values)
    return onehot_encoder

def get_onehot(labels, onehot_encoder):
    values = np.array(labels).reshape(-1, 1)
    onehot_encoded = onehot_encoder.transform(values)
    return onehot_encoded
    
def get_onehot_inverse(onehot_label, onehot_encoder):
    # invert first example
    if len(onehot_label.shape) == 1:
        onehot_label = onehot_label.reshape(1,-1)
    values = np.array(onehot_label)
    inverted = onehot_encoder.inverse_transform(onehot_label)
    return inverted

In [0]:
class AnimalsGenerator(keras.utils.Sequence) :
  def __init__(self, train_images, train_labels, batch_size) :
    self.train_images = train_images
    self.train_labels = train_labels
    self.batch_size = batch_size
    
    
  def __len__(self) :
    return int(np.floor(len(self.train_images) / self.batch_size))
  
  
  def __getitem__(self, idx) :
    batch_x = self.train_images[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.train_labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    return batch_x, batch_y

In [0]:
import h5py
hdf5_path = '/content/drive/My Drive/Colab Notebooks/animals/animals_224.h5'
hdf5_file = h5py.File(hdf5_path, "r")
train_images = hdf5_file["train_img"]
val_images = hdf5_file["val_img"]
test_images = hdf5_file["test_img"]

train_labels = hdf5_file["train_labels"]
val_labels = hdf5_file["val_labels"]
test_labels = hdf5_file["test_labels"]

onehot_encoder = get_onehot_encoder(train_labels)
train_labels= get_onehot(train_labels, onehot_encoder)


animals_generator = AnimalsGenerator(train_images, train_labels, 32)

val_labels = get_onehot(val_labels, onehot_encoder)
val_data = (val_images, val_labels)

test_labels = get_onehot(test_labels, onehot_encoder)


In [0]:
model = Sequential()

model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=10, activation="softmax"))

In [0]:
from keras.optimizers import SGD
opt = SGD(lr=0.001)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 56, 56, 256)      

In [0]:

from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
batch_size = 32
hist = model.fit_generator(animals_generator,steps_per_epoch=train_images.shape[0] // batch_size, epochs=10, validation_data= val_data, validation_steps=10,callbacks=[checkpoint,early])

print(hist.history)
acc_history = hist.history["acc"]
loss_history = hist.history['loss']
val_acc_history = hist.history['val_acc']
val_loss_history = hist.history['val_loss']


Epoch 1/10
654/654 [==============================] - 3890s 6s/step - loss: 0.3047 - acc: 0.9001 - val_loss: 0.2969 - val_acc: 0.9012

Epoch 00001: val_acc improved from -inf to 0.90122, saving model to vgg16_1.h5
Epoch 2/10
175/654 [=======>......................] - ETA: 46:09 - loss: 0.2953 - acc: 0.9009

In [0]:
import matplotlib.pyplot as plt
plt.plot(acc_history)
plt.plot(val_acc_history)
plt.plot(loss_history)
plt.plot(val_loss_history)
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()



In [0]:
from keras.models import load_model
import numpy as np

saved_model = load_model("vgg16_1.h5")

results = saved_model.evaluate(test_images,  test_labels, verbose=1, batch_size=128)
print('test loss, test acc:', results)

from sklearn.metrics import classification_report

y_pred = saved_model.predict(train_images, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_bool = np.argmax(train_labels, axis=1)
print('Train results:')
print(classification_report(y_bool, y_pred_bool))
print('-' * 100)

y_pred = saved_model.predict(test_images, batch_size=64, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
y_bool = np.argmax(test_labels, axis=1)
print('Test results:')
print(classification_report(y_bool, y_pred_bool))
print('-' * 100)

hdf5_file.close()

In [0]:
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model

img = image.load_img("pollo.jpg",target_size=(224,224))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)

saved_model = load_model("vgg16_1.h5")
output = saved_model.predict(img)

index = np.argmax(output, axis=1)
if index == 0:
  print("Perro")
elif index == 1:
  print("Caballo")
elif index == 2:
  print("Elefante")
elif index == 3:
  print("Mariposa")
elif index == 4:
  print("Gallina")
elif index == 5:
  print("Gato")
elif index == 6:
  print("Vaca")
elif index == 7:
  print("Oveja")
elif index == 8:
  print("Araña")
elif index == 9:
  print("Ardilla")
# 0 - Perro
# 1 - Caballo
# 2 - Elefante
# 3 - Mariposa
# 4 - Gallina
# 5 - Gato
# 6 - Vaca
# 7 - Oveja
# 8 - Araña
# 9 - Ardilla